#  Similarities
> Identifying the similarities between parties and members of the parliament based on their average voting behavior during roll call votes.

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

from bundestag import similarity as sim

In [ ]:
df = pd.read_parquet("../bundestag.de_votes.parquet")

## Voting outcomes by party

In [ ]:
party_votes = sim.get_votes_by_party(df)
sim.test_party_votes(party_votes)

In [ ]:
party_votes.head()

In [ ]:
party = "CDU/CSU"
px.scatter(
    data_frame=party_votes.loc[party_votes["Fraktion/Gruppe"] == party],
    x="date",
    y="fraction",
    color="vote",
)

## Member-Party similarity

A member of the German federal parliament is a Mitglied des Bundestages, so `mdb`.

In [ ]:
party_votes_pivoted = sim.pivot_party_votes_df(party_votes)
sim.test_party_votes_pivoted(party_votes_pivoted)
party_votes_pivoted.head()

In [ ]:
mdb = "Peter Altmaier"
mdb_votes = sim.prepare_votes_of_mdb(df, mdb)
sim.test_votes_of_mdb(mdb_votes)
mdb_votes.head()

In [ ]:
mdb_vs_parties = sim.align_mdb_with_parties(
    mdb_votes, party_votes_pivoted
).pipe(sim.compute_similarity, lsuffix="mdb", rsuffix="party")
sim.test_mdb_vs_parties(mdb_vs_parties)
mdb_vs_parties.head(3).T

In [ ]:
sim.plot_overall_similarity(
    mdb_vs_parties,
    x="Fraktion/Gruppe",
    title=f"Overall similarity of {mdb} with all parties",
);

Similarity over time

In [ ]:
sim.plot_similarity_over_time(
    mdb_vs_parties, "Fraktion/Gruppe", title=f"{mdb} vs time"
);

In [ ]:
sim.plot(
    mdb_vs_parties,
    title_overall=f"Overall similarity of {mdb} with all parties",
    title_over_time=f"{mdb} vs time",
)
plt.tight_layout()
plt.show()

## Party-Party similarity

In [ ]:
party_a = "SPD"
party_b = "BÜ90/GR"
partyA_vs_partyB = sim.align_party_with_party(
    party_votes_pivoted, party_a=party_a, party_b=party_b
).pipe(sim.compute_similarity, lsuffix="a", rsuffix="b")
sim.test_partyA_vs_partyB(partyA_vs_partyB)
partyA_vs_partyB.head(3).T

In [ ]:
party = "SPD"
partyA_vs_rest = sim.align_party_with_all_parties(
    party_votes_pivoted, party
).pipe(sim.compute_similarity, lsuffix="a", rsuffix="b")
sim.test_partyA_vs_partyB(partyA_vs_rest)
partyA_vs_rest.head(3).T

In [ ]:
sim.plot(
    partyA_vs_rest,
    title_overall=f"Overall similarity of {party} with all parties",
    title_over_time=f"{party} vs time",
    party_col="Fraktion/Gruppe_b",
)
plt.tight_layout()
plt.show()